In [ ]:
!pip uninstall -y torch torchvision torchaudio torch-geometric
!pip install --quiet torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu118
!pip install --quiet torch-geometric
!pip install --quiet ogb


Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 142.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 816.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.nn import GCNConv, SAGEConv, TransformerConv
from torch_geometric.utils import negative_sampling
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# -------------------------------
# Load Dataset (NO LEAKAGE)
# -------------------------------
dataset = PygLinkPropPredDataset('ogbl-ddi')
data = dataset[0]

split_edge = dataset.get_edge_split()

train_pos = split_edge['train']['edge'].to(device)
valid_pos = split_edge['valid']['edge'].to(device)
test_pos  = split_edge['test']['edge'].to(device)

num_nodes = data.num_nodes

# Construct graph using *only* training edges (IMPORTANT: prevents leakage)
data.edge_index = train_pos.t().contiguous().to(device)

evaluator = Evaluator(name='ogbl-ddi')

Using device: cuda


Downloaded 0.04 GB: 100%|██████████| 46/46 [00:06<00:00,  7.64it/s]


Extracting dataset/ddi.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 50.14it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 3258.98it/s]

Saving...



Done!
/usr/local/lib/python3.12/dist-packages/ogb/linkproppred/dataset_pyg.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.pro

In [ ]:
class BaseModel(nn.Module):
    def decode(self, z, edge):
        src, dst = edge[:, 0], edge[:, 1]
        return (z[src] * z[dst]).sum(dim=1)

In [ ]:
class GCN(BaseModel):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv1 = GCNConv(hidden_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def encode(self, edge_index):
        # Random non-trainable node features → prevents memorization
        x = torch.randn(num_nodes, self.hidden_dim, device=device)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [ ]:
class GraphSAGE(BaseModel):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv1 = SAGEConv(hidden_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)

    def encode(self, edge_index):
        x = torch.randn(num_nodes, self.hidden_dim, device=device)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x


In [ ]:
class GraphTransformer(BaseModel):
    def __init__(self, hidden_dim, heads=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv1 = TransformerConv(hidden_dim, hidden_dim // heads, heads=heads)
        self.conv2 = TransformerConv(hidden_dim, hidden_dim // heads, heads=heads)

    def encode(self, edge_index):
        x = torch.randn(num_nodes, self.hidden_dim, device=device)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [ ]:
def get_loss(model, edge_index, pos_edges):
    z = model.encode(edge_index)
    pos_score = model.decode(z, pos_edges)

    neg_edges = negative_sampling(
        edge_index=edge_index,
        num_nodes=num_nodes,
        num_neg_samples=pos_edges.size(0)
    ).to(device)

    neg_score = model.decode(z, neg_edges)

    # Margin ranking hinge loss
    loss = -torch.log(torch.sigmoid(pos_score)).mean() - torch.log(1 - torch.sigmoid(neg_score)).mean()
    return loss


In [ ]:
def evaluate(model):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.edge_index)

        # Positive scores
        pos_scores = model.decode(z, test_pos).view(-1).cpu()

        # Negative sampling + batch to avoid OOM
        neg_test = negative_sampling(
            edge_index=data.edge_index.cpu(),
            num_nodes=num_nodes,
            num_neg_samples=test_pos.size(0)
        )
        neg_test = neg_test.t().to(device)

        neg_scores_list = []
        batch = 200000  # safe chunk size
        for i in range(0, neg_test.size(0), batch):
            chunk = neg_test[i:i+batch]
            neg_scores_list.append(model.decode(z, chunk).view(-1).cpu())

        neg_scores = torch.cat(neg_scores_list)

        result = evaluator.eval({
            'y_pred_pos': pos_scores,
            'y_pred_neg': neg_scores,
        })

        return result['hits@20']

In [ ]:
def train_model(name, model, epochs=1000, lr=0.001):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, epochs + 1):
        model.train()
        optimizer.zero_grad()

        z = model.encode(data.edge_index)
        neg_samples = negative_sampling(
            edge_index=data.edge_index,
            num_nodes=num_nodes,
            num_neg_samples=train_pos.size(0)
        ).t().to(device)

        pos_out = model.decode(z, train_pos).sigmoid()
        neg_out = model.decode(z, neg_samples).sigmoid()

        loss = -torch.log(pos_out + 1e-15).mean() - torch.log(1 - neg_out + 1e-15).mean()
        loss.backward()
        optimizer.step()

        if epoch % 5 == 0:
            print(f"{name} Epoch {epoch}/{epochs} - Loss = {loss.item():.4f}")

    hits20 = evaluate(model)
    print(f"{name}: Hits@20 = {hits20:.4f}")
    return hits20

In [ ]:
gcn_hits  = train_model("GCN", GCN(128))
sage_hits = train_model("GraphSAGE", GraphSAGE(128))
gt_hits   = train_model("GraphTransformer", GraphTransformer(128))

print("\n==== FINAL RESULTS ====")
print(f"GCN Hits@20       = {gcn_hits:.4f}")
print(f"GraphSAGE Hits@20 = {sage_hits:.4f}")
print(f"Transformer Hits@20 = {gt_hits:.4f}")

GCN Epoch 5/1000 - Loss = 4.0406
GCN Epoch 10/1000 - Loss = 1.8138
GCN Epoch 15/1000 - Loss = 2.3580
GCN Epoch 20/1000 - Loss = 1.7851
GCN Epoch 25/1000 - Loss = 1.4486
GCN Epoch 30/1000 - Loss = 1.6299
GCN Epoch 35/1000 - Loss = 1.4335
GCN Epoch 40/1000 - Loss = 1.4723
GCN Epoch 45/1000 - Loss = 1.4954
GCN Epoch 50/1000 - Loss = 1.8235
GCN Epoch 55/1000 - Loss = 1.4470
GCN Epoch 60/1000 - Loss = 1.4004
GCN Epoch 65/1000 - Loss = 1.3768
GCN Epoch 70/1000 - Loss = 1.4690
GCN Epoch 75/1000 - Loss = 1.4660
GCN Epoch 80/1000 - Loss = 1.6700
GCN Epoch 85/1000 - Loss = 1.4468
GCN Epoch 90/1000 - Loss = 1.3983
GCN Epoch 95/1000 - Loss = 1.5373
GCN Epoch 100/1000 - Loss = 1.5461
GCN Epoch 105/1000 - Loss = 1.3005
GCN Epoch 110/1000 - Loss = 1.3751
GCN Epoch 115/1000 - Loss = 1.3616
GCN Epoch 120/1000 - Loss = 1.4850
GCN Epoch 125/1000 - Loss = 1.4569
GCN Epoch 130/1000 - Loss = 1.3237
GCN Epoch 135/1000 - Loss = 1.3722
GCN Epoch 140/1000 - Loss = 1.7339
GCN Epoch 145/1000 - Loss = 1.3500
GCN E

In [ ]:
@torch.no_grad()
def debug_evaluator(model):
    model = model.to(device)                # ✅ move model to cuda
    model.eval()

    z = model.encode(data.edge_index)       # now both live on GPU

    pos_scores = model.decode(z, test_pos).view(-1).cpu()

    neg_test = negative_sampling(
        edge_index=data.edge_index.cpu(),   # ✅ negative_sampling works on CPU
        num_nodes=num_nodes,
        num_neg_samples=test_pos.size(0),
    )
    neg_test = neg_test.to(device)          # ✅ then move back to CUDA
    neg_scores = model.decode(z, neg_test).view(-1).cpu()

    result = evaluator.eval({
        'y_pred_pos': pos_scores,
        'y_pred_neg': neg_scores,
    })

    print("Returned evaluator keys:", result.keys())
    return result

debug_evaluator(GCN(32))


Returned evaluator keys: dict_keys(['hits@20'])


{'hits@20': 1.0}